# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [1]:
%pip install iso3166

Note: you may need to restart the kernel to use updated packages.


### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
%pip install --upgrade plotly

     |████████████████████████████████| 15.2 MB 5.5 MB/s eta 0:00:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.9.0
    Uninstalling plotly-5.9.0:
      Successfully uninstalled plotly-5.9.0


### Import Statements

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [19]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [4]:
df_data.shape

(4324, 9)

It has 4324 rows and 9 columns

In [10]:
df_data.isna().values.any()

True

In [11]:
df_data.duplicated().values.any()

False

Has na values but no duplicates

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [20]:
clean_data = df_data.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

In [22]:
clean_data = clean_data.dropna()

## Descriptive Statistics

In [23]:
clean_data.describe

<bound method NDFrame.describe of       Organisation                                           Location  \
0           SpaceX         LC-39A, Kennedy Space Center, Florida, USA   
1             CASC  Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...   
3        Roscosmos       Site 200/39, Baikonur Cosmodrome, Kazakhstan   
4              ULA           SLC-41, Cape Canaveral AFS, Florida, USA   
5             CASC       LC-9, Taiyuan Satellite Launch Center, China   
...            ...                                                ...   
3855  US Air Force            SLC-4W, Vandenberg AFB, California, USA   
3971  US Air Force           SLC-20, Cape Canaveral AFS, Florida, USA   
3993  US Air Force           SLC-20, Cape Canaveral AFS, Florida, USA   
4000  US Air Force           SLC-20, Cape Canaveral AFS, Florida, USA   
4020  US Air Force           SLC-20, Cape Canaveral AFS, Florida, USA   

                            Date  \
0     Fri Aug 07, 2020 05:12 UTC   
1     Thu Aug 06,

In [24]:
clean_data.describe()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
count,963,963,963,963,963,963,963
unique,25,56,963,962,2,56,4
top,CASC,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",H-IIA 202 | IGS-Radar 5,StatusActive,450.0,Success
freq,157,120,1,2,585,136,909


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [26]:
by_org = clean_data.groupby('Organisation')

In [32]:
by_org.count().sort_values('Location',ascending=False)

,Location,Date,Detail,Rocket_Status,Price,Mission_Status
Organisation,,,,,,
CASC,157,157,157,157,157,157
NASA,149,149,149,149,149,149
SpaceX,99,99,99,99,99,99
ULA,98,98,98,98,98,98
Arianespace,96,96,96,96,96,96
Northrop,83,83,83,83,83,83
ISRO,67,67,67,67,67,67
MHI,37,37,37,37,37,37
VKS RF,33,33,33,33,33,33


In [43]:
fig = px.pie(by_org.count().sort_values('Location',ascending=False), values=by_org.count().sort_values('Location',ascending=False).Location, 
             names=by_org.count().sort_values('Location',ascending=False).index,
            title='Launches per Company') 
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [46]:
active_rockets = clean_data.groupby('Rocket_Status')

In [47]:
active_rockets.count()

,Organisation,Location,Date,Detail,Price,Mission_Status
Rocket_Status,,,,,,
StatusActive,585,585,585,585,585,585
StatusRetired,378,378,378,378,378,378


# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [51]:
clean_data.groupby('Mission_Status').count()

,Organisation,Location,Date,Detail,Rocket_Status,Price
Mission_Status,,,,,,
Failure,36,36,36,36,36,36
Partial Failure,17,17,17,17,17,17
Prelaunch Failure,1,1,1,1,1,1
Success,909,909,909,909,909,909


AttributeError: 'Series' object has no attribute 'summary'

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [63]:
clean_data.Price = clean_data.Price.str.replace(',','')

In [64]:
clean_data.Price = pd.to_numeric(clean_data.Price)

In [66]:
fig = px.histogram(clean_data, x=clean_data.Price)
fig.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [116]:
clean_data['Location'] = clean_data['Location'].astype(str)
loc = clean_data['Location'].str.split(',')

clean_data['country'] = loc.str[-1]

In [130]:
clean_data.loc[(clean_data['country'] == ' Yellow Sea'),'country'] = 'China'

In [132]:
clean_data.loc[(clean_data['country'] == ' New Mexico'),'country'] = 'USA'
clean_data.loc[(clean_data['country'] == ' Shahrud Missile Test Site'),'country'] = 'Iran'
clean_data.loc[(clean_data['country'] == ' Pacific Missile Range Facility'),'country'] = 'USA'
clean_data.loc[(clean_data['country'] == ' Barents Sea'),'country'] = 'Russia Federation'


In [147]:
clean_data.loc[(clean_data['country'] == 'Russia'),'country'] = 'Russian Federation'
clean_data.loc[(clean_data['country'] == ' Gran Canaria'),'country'] = 'USA'

In [148]:
clean_data.country = clean_data.country.str.strip()
clean_data.country.unique()

array(['USA', 'China', 'Kazakhstan', 'Japan', 'New Zealand',
       'Russian Federation', 'France', 'India'], dtype=object)

In [158]:
clean_data.country = clean_data.country.astype('string')
clean_data.country

0              USA
1            China
3       Kazakhstan
4              USA
5            China
           ...    
3855           USA
3971           USA
3993           USA
4000           USA
4020           USA
Name: country, Length: 963, dtype: string

In [160]:
clean_data['ISO'] = countries.get(clean_data.country)[2]

AttributeError: 'Series' object has no attribute 'upper'

# Use a Choropleth Map to Show the Number of Failures by Country


In [149]:
countries.get('Russian Federation')

Country(name='Russian Federation', alpha2='RU', alpha3='RUS', numeric='643', apolitical_name='Russian Federation')

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

# Analyse the Amount of Money Spent by Organisation per Launch

# Chart the Number of Launches per Year

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 